<a href="https://colab.research.google.com/github/Ajwebdevs/Translartor_Hackthon/blob/main/Translator_model_MT5base_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Machine learning model using MT5_base


*machine learning model using MT5_base from hugging face*



#*Installing the transformers library*

In [1]:
!pip install transformers sentencepiece datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00


#loading all the needed dependancies

In [2]:
from datasets import load_dataset
from IPython.display import display
from google.colab import drive
from IPython.html import widgets
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import torch
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm_notebook

/usr/local/lib/python3.10/dist-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [3]:
#getting to know the max_var
#model.config.max_length

# Loading the pre-trained google mt5 model

In [4]:
model_repo = 'google/mt5-large'
#defining the max_varible ( source stack overflow ) i dint know this had to be done
max_seq_len = 20

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_repo)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


# Getting the model


In [6]:
#getting model
model = AutoModelForSeq2SeqLM.from_pretrained(model_repo)
model = model.cuda()

In [7]:
#basic tokenisation for test_01 to be delted after completion Test batch 01
input_sent = "Welcome to the team hexa"
tokens_id = tokenizer.encode(input_sent, return_tensors = 'pt').cuda()
tokens_id

#sending to the model
model_output = model.generate(tokens_id)
print(model_output)

text_output = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(model_output[0]))
print(text_output)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[     0, 250099,    259,  29230,    288,    287,   4644, 134844,    259,
          29230,    288,    287,   4644,    259,  29230,    288,    287,   4644,
         134844,    259]], device='cuda:0')
<pad> <extra_id_0> Welcome to the team hexa Welcome to the team Welcome to the team hexa 


#loading the models

In [8]:
beta_test_input_str = '<hi>Hello world!, this is a demo test script'
input_id = tokenizer.encode(beta_test_input_str, return_tensors='pt')
print('Input IDs:', input_id)

tokens = tokenizer.convert_ids_to_tokens(input_id[0])
print('Tokens:', tokens)

Input IDs: tensor([[  1042,   1500,    669, 141528,   4836,    309,    261,    714,    339,
            259,    262,  57273,   2978,  42817,      1]])
Tokens: ['▁<', 'hi', '>', 'Hello', '▁world', '!', ',', '▁this', '▁is', '▁', 'a', '▁demo', '▁test', '▁script', '</s>']


#alt dataset loading

In [9]:
#loading the alt dataset
dataset = load_dataset('alt')

Generating train split:   0%|          | 0/18088 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1019 [00:00<?, ? examples/s]

#Train and test dataset split

In [10]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [11]:
#printing out the test dataset with almsot 1019 data
test_dataset

Dataset({
    features: ['SNT.URLID', 'SNT.URLID.SNTID', 'url', 'translation'],
    num_rows: 1019
})

In [12]:
#printing out the test dataset with almsot 18088 data
train_dataset

Dataset({
    features: ['SNT.URLID', 'SNT.URLID.SNTID', 'url', 'translation'],
    num_rows: 18088
})

#tokken languge mapping


In [13]:
LANG_TOKKEN_MAPPING = {
    'hi': '<hi>',
    'en': '<en>'
}

#implementing the tokenizer according to the documentation described



In [14]:
sp_tokens_dictate = {'additional_special_tokens': list(LANG_TOKKEN_MAPPING.values())}
tokenizer.add_special_tokens(sp_tokens_dictate) #dictating the special tokkens
model.resize_token_embeddings(len(tokenizer))
#arpitha this may overload the gpu, close all the background apps before running the cell or as a pyblock on pycharm


You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 250102. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(250102, 1024)

#truncating the max text length to 20 vars , max limit anything above would be cutoff

In [15]:
tokens_id = tokenizer.encode(
    beta_test_input_str, return_tensors='pt',
    padding='max_length',
    truncation=True, max_length=max_seq_len
)
print(tokens_id)
# padding tokken is gonna show 0 where thee charachter limit is not reached

tensor([[250100,  30273,   4836,    309,    261,    714,    339,    259,    262,
          57273,   2978,  42817,      1,      0,      0,      0,      0,      0,
              0,      0]])


#tokken mapping and encoder-decoder model

In [16]:
def encode_input_str(text, target_lang, tokenizer, seq_len,
                     lang_token_map=LANG_TOKKEN_MAPPING):
  target_lang_token = lang_token_map[target_lang]

  # Tokenize and add special token
  input_ids = tokenizer.encode(
      text = target_lang_token + text,
      return_tensors = 'pt',
      padding = 'max_length',
      turncation = True,
      max_length = seq_len)

  return input_ids[0]

  def encode_target_str(text, tokenizer, seq_len,
                        lang_token_map=LANG_TOKKEN_MAPPING):
    tokens_id = tokenizer.encode(
        text = text,
        return_tensors = 'pt',
        padding = 'max_length',
        truncation = True,
        max_length = seq_len)

    return tokens_id[0]

def format_translation_data(translations, lang_token_map,
                            tokenizer, seq_len=128):
  #LANGUAGE CHOSER
  langs = list(lang_token_map.keys())
  input_lang, target_lang = np.random.choice(langs, size= 2, replace=False)

  #getting the target language
  input_text = translations[input_lang]
  output_text = translations[target_lang]